<a href="https://colab.research.google.com/github/WengGeyu/NMA-DL/blob/main/load_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# !pip3 install http://download.pytorch.org/whl/cu80/torch-0.4.0-cp36-cp36m-linux_x86_64.whl
# !pip3 install torchvision
# !pip3 install numpy==1.14.1

# Import all the necessary packages
import numpy as np
import nibabel as nib
# import itk
# import itkwidgets
from ipywidgets import interact, interactive, IntSlider, ToggleButtons
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')

from collections import defaultdict
import time
import os

from tqdm import tqdm

import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import autograd, optim
import torch.backends.cudnn as cudnn
import torch.nn as nn

import torchvision.utils as vutils

cudnn.benchmark = True
import matplotlib.pyplot as plt
%matplotlib inline

NameError: ignored

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# !pip install openneuro-py
# import openneuro as on
# on.download(dataset='ds003949', target_dir='/content/drive/MyDrive/Colab Notebooks')

In [5]:
input_path = "/content/drive/MyDrive/Colab Notebooks/Aneurysm_Detection-main/dataset/derivatives/N4_bias_field_corrected/sub-000/ses-20110101/anat"
input_filename = os.path.join(input_path,'sub-000_ses-20110101_desc-angio_N4bfc_mask.nii.gz')
image_obj = nib.load(input_filename)
print(f'Type of the image {type(image_obj)}')

Type of the image <class 'nibabel.nifti1.Nifti1Image'>


In [ ]:
# Extract data as numpy ndarray
image_data = image_obj.get_fdata()
# Get the image shape and print it out
height, width, depth = image_data.shape
print(f"The image object has the following dimensions: height: {height}, width:{width}, depth:{depth}")
# Select random layer number
maxval = depth
i = np.random.randint(0, maxval)
# Define a channel to look at
channel = 0
print(f"Plotting Layer {i} Channel {channel} of Image")
plt.imshow(image_data[:, :, i], cmap='gray')
plt.axis('off');

In [13]:
# os.getcwd()
os.chdir(/content/drive/MyDrive/Colab Notebooks/Aneurysm_Detection-main/dataset_creation)
#create_dataset_patches_neg_and_pos.py --config config_creation_ds_patches.json

SyntaxError: ignored

In [ ]:
!git clone https://github.com/MIC-DKFZ/nnUNet.git
%cd nnUNet
!pip install -e .
# nnUNetv2_plan_and_preprocess -d DATASET_ID --verify_dataset_integrity

SyntaxError: ignored